In [1]:
from process.preproc_clinic import writeClinic
from process.preproc_img import imgRedir, mvRaw, preprocFSL, preprocCAT12
from process.preproc_combine import genHCData, combinePatData
from src.feature.base import genROI
from src.feature.texture import genTextureFeature, dropByCorrelation
from src.utils.data import getPandas, getConfig, writeConfig, writePandas
from src.model.stats import stats_analyze
import os
import numpy as np
import pandas as pd

In [2]:
# Root directory as working directory
os.chdir('..')
os.getcwd()

'/home/biobot/disk/ldopa'

## Preprocessing & Statistical Analysis

In [3]:
# Get Image Data
imgRedir()
# Get Clinical Data
writeClinic()
# Combine Data & Generate HC Data
combinePatData()
#genHCData()
# Generate Subject Directory & Move Raw Image Data
#mvRaw(getPandas('img_raw'))
# Now all raw image data and clinical data are set up

/home/biobot/disk/ldopa/code/process/preproc_clinic.py:11: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_u3_score = pd.read_csv(os.path.join('data', 'csv', 'MDS_UPDRS_Part_III_CAL.csv'))


,PATNO,EVENT_ID,INFODT,NUPDR3OF,NUPDR3ON,IMG_ID,AGE,SEX,ORIG_ENTRY,SCORE,...,Mfg Model,Field Strength,Group,Modality,Description,Type,Acq Date,Format,Downloaded,IMG_ROOT
0,3107,V04,2012-03-01,2,2,I296431,70.6,1,03/2011,0.000000,...,TrioTim,3.0,PD,MRI,MPRAGE GRAPPA,Original,3/28/2012,DCM,10/24/2022,data/subj/3107V04I296431
1,3107,V06,2013-05-01,18,6,I378218,71.7,1,03/2011,0.666667,...,TrioTim,3.0,PD,MRI,MPRAGE GRAPPA,Original,5/15/2013,DCM,10/24/2022,data/subj/3107V06I378218
2,3107,V10,2015-05-01,19,6,I498876,73.7,1,03/2011,0.684211,...,TrioTim,3.0,PD,MRI,Sag MPRAGE GRAPPA,Original,5/08/2015,DCM,10/24/2022,data/subj/3107V10I498876
3,3108,V06,2013-04-01,13,15,I378222,51.8,0,04/2011,-0.153846,...,TrioTim,3.0,PD,MRI,MPRAGE GRAPPA,Original,4/24/2013,DCM,10/24/2022,data/subj/3108V06I378222
4,3108,V10,2015-05-01,22,12,I498885,53.8,0,04/2011,0.454545,...,TrioTim,3.0,PD,MRI,Sag MPRAGE GRAPPA,Original,5/06/2015,DCM,10/24/2022,data/subj/3108V10I498885
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,101179,V04,2022-04-01,45,25,I1571515,45.0,0,03/2021,0.444444,...,Achieva dStream,3.0,PD,MRI,3D T1-weighted,Original,4/04/2022,DCM,10/24/2022,data/subj/101179V04I1571515
351,101187,V04,2022-06-01,18,7,I1616582,64.0,1,09/2022,0.611111,...,SIGNA Architect,3.0,PD,MRI,SAG 3D T1 FSPGR,Original,6/10/2022,DCM,10/24/2022,data/subj/101187V04I1616582
352,101295,V04,2022-08-01,31,23,I1616170,66.3,1,04/2021,0.258065,...,DISCOVERY MR750,3.0,PD,MRI,"T1-weighted, 3D VOLUMETRIC",Original,8/04/2022,DCM,10/24/2022,data/subj/101295V04I1616170
353,140041,BL,2022-03-01,22,9,I1575193,55.6,0,02/2022,0.590909,...,SIGNA Architect,3.0,PD,MRI,3D T1-weighted,Original,4/01/2022,DCM,10/24/2022,data/subj/140041BLI1575193


In [ ]:
# Image Preprocessing
# For Patient Data:
preprocFSL('pat_data')
preprocCAT12('pat_data')
# For HC Data:
preprocFSL('hc_data')
preprocCAT12('hc_data')

# Time Consuming!!!
# All volume data should be saved automatically

In [73]:
# Dataset Split
# One patient may have multiple images, so we need to split the dataset by patient
from sklearn.model_selection import GroupShuffleSplit

data = getPandas('pat_data')
data_conf = getConfig('data')

pat = data.sample(frac=1, random_state=1).drop_duplicates(subset=['PATNO', 'EVENT_ID'], keep='first')
pat = pat.drop(pat[pat['NUPDR3OF'] < 5].index)
pat = pat.drop(pat[pat['LEDD'] > 5000].index)
pat = pat.reset_index(drop=False)

splitter = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=1)
split = splitter.split(pat, groups=pat['PATNO'])
train_inds, test_inds = next(split)
train_inds = pat.iloc[train_inds]['index'].values
test_inds = pat.iloc[test_inds]['index'].values

In [37]:
# ratio = len(data[data['CAT_MDS']==1]) / len(data)
# test_size = int(len(data) * 0.18)
# unique_rec = data[~data.duplicated(subset=['PATNO'], keep=False)]
# unique_pos = unique_rec[unique_rec['CAT_MDS']==1].reset_index(drop=False)
# unique_neg = unique_rec[unique_rec['CAT_MDS']==0].reset_index(drop=False)
# test_pos_inds = unique_pos.sample(n=int(test_size * ratio), random_state=0)['index'].values
# test_neg_inds = unique_neg.sample(n=test_size-int(test_size * ratio), random_state=0)['index'].values
# test_inds = np.concatenate([test_pos_inds, test_neg_inds])
# train_inds = np.setdiff1d(data.index.values, test_inds)

In [74]:
# Data Analysis
group = data_conf['data_group']
x = data[group['demo'] + group['clinic'] + ['PATNO']]
y = data[['CAT_MDS']]
x_clinic_train = x.iloc[train_inds].reset_index(drop=True)
x_clinic_test = x.iloc[test_inds].reset_index(drop=True)
y_train = y.iloc[train_inds].reset_index(drop=True)
y_test = y.iloc[test_inds].reset_index(drop=True)
stats_analyze(x_clinic_train, x_clinic_test, y_train, y_test, data_conf, print)

SEX chi2 p: 0.5607707690730641
CAT_MDS chi2 p: 0.8185018508515416
AGE Normaltest p_train: [0.09196485], p_test: [0.35266262]
AGE t-test p: Ttest_indResult(statistic=array([0.95625181]), pvalue=array([0.34001405]))
NUPDR3OF Normaltest p_train: [0.03611899], p_test: [0.27855476]
NUPDR3OF ranksums p: RanksumsResult(statistic=array([0.31434421]), pvalue=array([0.75325962]))
LEDD Normaltest p_train: [1.31142969e-14], p_test: [0.01990032]
LEDD ranksums p: RanksumsResult(statistic=array([-1.51286513]), pvalue=array([0.13031392]))
DURATION Normaltest p_train: [0.03528692], p_test: [0.19010019]
DURATION ranksums p: RanksumsResult(statistic=array([-0.59792282]), pvalue=array([0.54989143]))



In [75]:
print('Number of CAT_MDS = 0 in train set: {}'.format(y_train['CAT_MDS'].value_counts()[0]))
print('Number of CAT_MDS = 0 in test set: {}'.format(y_test['CAT_MDS'].value_counts()[0]))
print('Number of CAT_MDS = 1 in train set: {}'.format(y_train['CAT_MDS'].value_counts()[1]))
print('Number of CAT_MDS = 1 in test set: {}'.format(y_test['CAT_MDS'].value_counts()[1]))
print('Ratio of CAT_MDS = 0 in train set: {}'.format(y_train['CAT_MDS'].value_counts()[0] / len(y_train)))
print('Ratio of CAT_MDS = 0 in test set: {}'.format(y_test['CAT_MDS'].value_counts()[0] / len(y_test)))

Number of CAT_MDS = 0 in train set: 77
Number of CAT_MDS = 0 in test set: 18
Number of CAT_MDS = 1 in train set: 97
Number of CAT_MDS = 1 in test set: 26
Ratio of CAT_MDS = 0 in train set: 0.4425287356321839
Ratio of CAT_MDS = 0 in test set: 0.4090909090909091


In [76]:
# Save train/test indices to data config
data_conf['indices'] = {
    'pat': {}
}
data_conf['indices']['pat'] = {
    'train': train_inds.tolist(),
    'test': test_inds.tolist()
}
writeConfig('data', data_conf)

## Feature Extraction

In [3]:
## Texture Features
# Generate ROI
#genROI()
# Generate Texture Features
genTextureFeature('pat_data', 'ANTs_Reg')

Calculating radiomic features for data/subj/3107V04I296431/fsl/reg.nii.gz...
Calculating radiomic features for data/subj/3107V06I378218/fsl/reg.nii.gz...
Calculating radiomic features for data/subj/3107V10I498876/fsl/reg.nii.gz...
Calculating radiomic features for data/subj/3108V06I378222/fsl/reg.nii.gz...
Calculating radiomic features for data/subj/3108V10I498885/fsl/reg.nii.gz...
Calculating radiomic features for data/subj/3113V06I393646/fsl/reg.nii.gz...
Calculating radiomic features for data/subj/3116V04I366133/fsl/reg.nii.gz...
Calculating radiomic features for data/subj/3118V10I665283/fsl/reg.nii.gz...
Calculating radiomic features for data/subj/3118V10I665282/fsl/reg.nii.gz...
Calculating radiomic features for data/subj/3120V10I738217/fsl/reg.nii.gz...
Calculating radiomic features for data/subj/3120V10I738218/fsl/reg.nii.gz...
Calculating radiomic features for data/subj/3123V10I757784/fsl/reg.nii.gz...
Calculating radiomic features for data/subj/3123V10I757789/fsl/reg.nii.gz...

In [3]:
data = getPandas('pat_data')

In [5]:
data['AGE_CORRECTED_SGM'] = data['IMG_ROOT'] + '/agecorrected.nii'

In [21]:
data['ANTs_Reg'] = data['IMG_ROOT'] + os.sep + 'fsl' + os.sep + 'reg.nii.gz'
data['FSL_GM'] = data['IMG_ROOT'] + os.sep + 'fsl' + os.sep + 'reg_gm.nii'
data['FSL_WM'] = data['IMG_ROOT'] + os.sep + 'fsl' + os.sep + 'reg_wm.nii'
data['FSL_CSF'] = data['IMG_ROOT'] + os.sep + 'fsl' + os.sep + 'reg_csf.nii'
data['FSL_SGM'] = data['IMG_ROOT'] + os.sep + 'fsl' + os.sep + 'sreg_gm_masked.nii'
data['CAT12_GM'] = data['IMG_ROOT'] + os.sep + 'cat12' + os.sep + 'mri' + os.sep + 'mwp1raw.nii'
data['CAT12_SGM'] = data['IMG_ROOT'] + os.sep + 'cat12' + os.sep + 'mri' + os.sep + 'smwp1raw_masked.nii'
data['CAT12_WM'] = data['IMG_ROOT'] + os.sep + 'cat12' + os.sep + 'mri' + os.sep + 'mwp2raw.nii'
data['CAT12_CSF'] = data['IMG_ROOT'] + os.sep + 'cat12' + os.sep + 'mri' + os.sep + 'mwp3raw.nii'
from xml.dom import minidom
report_list = list(data['IMG_ROOT'] + os.sep + 'cat12' + os.sep + 'report' + os.sep + 'cat_raw.xml')
vol_list = []
for report in report_list:
    root = minidom.parse(report).documentElement
    tiv_str = root.getElementsByTagName('subjectmeasures')[1].getElementsByTagName('vol_TIV')[0].childNodes[0].data
    vol_str = root.getElementsByTagName('subjectmeasures')[1].getElementsByTagName('vol_abs_CGW')[0].childNodes[0].data
    tiv = float(tiv_str)
    gm = float(vol_str.split(' ')[1])
    wm = float(vol_str.split(' ')[2])
    vol_list.append({'TIV': tiv, 'GM_VOL': gm, 'WM_VOL': wm})
vol_list = pd.DataFrame(vol_list)
data = pd.concat([data, vol_list], axis=1)

In [7]:
from src.utils.data import writePandas
writePandas('pat_data', data)

In [3]:
dropByCorrelation('pat_data', 'pat_ANTs_Reg_radiomic', 'CAT', 0.8)

lRN
lSN
lSTN
lCAU
lPUT
lGPe
lGPi
lTHA
rRN
rSN
rSTN
rCAU
rPUT
rGPe
rGPi
rTHA


<Figure size 640x480 with 0 Axes>